# K-medoides

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
# Instalación de las librerias necesarias
!pip install scipy --upgrade --user
!pip install numpy --upgrade --user
!pip install pandas --upgrade --user
#Distancia de Gower para saber las encuestas más cercanas
!pip install gower
!pip install mlxtend

     |████████████████████████████████| 25.9MB 170kB/s 
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 14.6MB 316kB/s 
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.0.5)
  Created wheel for gower: filename=gower-0.0.5-cp36-none-any.whl size=4232 sha256=34171773b070bc0aaf06dfc7f927c9bd907204491997a2e629af26b25a1c1579
  Stored in directory: /root/.cache/pip/wheels/c0/09/9b/072d54d6ced0f43a179852e3f09532d0131e25ff7cb4e5ee75
Successfully built gower


In [5]:
# Importación de las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
from sklearn import datasets
import gower
import time

## Funciones de k-medoides

In [ ]:
#tomado de https://towardsdatascience.com/k-medoids-clustering-on-iris-data-set-1931bf781e05
def init_medoids(X, k):
  from numpy.random import choice
  from numpy.random import seed
  seed(1)
  samples = choice(len(X), size = k, replace = False)
  return X[samples, :]

In [ ]:
def compute_d_p(X, medoids, p):
    m = len(X)
    medoids_shape = medoids.shape
    # If a 1-D array is provided, 
    # it will be reshaped to a single row 2-D array
    if len(medoids_shape) == 1: 
        medoids = medoids.reshape((1,len(medoids)))
    k = len(medoids)
    
    S = np.empty((m, k))
    
    for i in range(m):
        d_i = np.linalg.norm(X[i, :] - medoids, ord=p, axis=1)
        S[i, :] = d_i**p

    return S

In [ ]:
#Función para calcular distancias bajo el método de gower
def compute_gower(X, medoids):
    m = len(X)
    medoids_shape = medoids.shape
    if len(medoids_shape) == 1: 
      medoids = medoids.reshape((1,len(medoids)))
    k = len(medoids)
    S = np.empty((m, k))
    for j in range(k):
      d_i = gower.gower_topn(medoids[[j],:], X, n = len(X))
      df = pd.DataFrame(d_i)
      df = df.sort_values('index', ignore_index=True)
      S[:,j] = df['values']
    return S

In [ ]:
def assign_labels(S):
    return np.argmin(S, axis=1)

In [ ]:
def update_medoids(points, medoids):    
    S = compute_gower(points, medoids)
    labels = assign_labels(S)        
    out_medoids = medoids
                
    for i in set(labels):        
        avg_dissimilarity = np.sum(compute_gower(points, medoids[i]))
        cluster_points = points[labels == i]
        
        for datap in cluster_points:
            new_medoid = datap
            new_dissimilarity= np.sum(compute_gower(points, datap))
            if new_dissimilarity < avg_dissimilarity :
                avg_dissimilarity = new_dissimilarity                
                out_medoids[i] = datap                
    return out_medoids

In [ ]:
def has_converged(old_medoids, medoids):
    return set([tuple(x) for x in old_medoids]) == set([tuple(x) for x in medoids])

In [ ]:
#tomado de https://towardsdatascience.com/k-medoids-clustering-on-iris-data-set-1931bf781e05
def kmedoids(X, k, starting_medoids=None, max_steps=np.inf):
    if starting_medoids is None:
        medoids = init_medoids(X, k)
    else:
        medoids = starting_medoids
        
    converged = False
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_medoids = medoids.copy()        
        S = compute_gower(X, medoids)        
        labels = assign_labels(S)        
        medoids = update_medoids(X, medoids)        
        converged = has_converged(old_medoids, medoids)
        i += 1
    return (medoids,labels)

In [ ]:
def mark_matches(a, b, exact=False):
    assert a.shape == b.shape
    a_int = a.astype(dtype=int)
    b_int = b.astype(dtype=int)
    all_axes = tuple(range(len(a.shape)))
    assert ((a_int == 0) | (a_int == 1) | (a_int == 2) | (a_int == 3) | (a_int == 4) | (a_int == 5)).all()
    assert ((b_int == 0) | (b_int == 1) | (b_int == 2) | (b_int == 3) | (b_int == 4) | (b_int == 5)).all()
    
    exact_matches = (a_int == b_int)
    if exact:
        return exact_matches

    assert exact == False
    num_exact_matches = np.sum(exact_matches)
    if (2*num_exact_matches) >= np.prod (a.shape):
        return exact_matches
    return exact_matches == False

In [ ]:
def count_matches(a, b, exact=False):
    matches = mark_matches(a, b, exact=exact)
    return np.sum(matches)

## Ejecución de k-medoides

In [6]:
df_dicc = pd.read_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Modeling/T2_ECV_condum_sinout_ord.csv', sep=';')

In [ ]:
#Código para leerlo internamente en JUPYTER
#df = pd.read_csv('T2_ECV_condum_sinout_ord.csv', sep=';')
#df_dicc.info()
df.columns

Index(['Unnamed: 0', 'p_5_1', 'p_5_2', 'p_6_1', 'p_6_2', 'p_6_3', 'p_6_4',
       'p_6_5', 'p_6_6', 'p_6_7',
       ...
       'p_314', 'p_315', 'p_316', 'p_317', 'p_318', 'p_319', 'p_320', 'p_321',
       'p_322', 'p_323'],
      dtype='object', length=489)

In [ ]:
df1 = pd.DataFrame(df)
df_dicc = df.drop('Unnamed: 0', axis=1)
df_dicc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9040 entries, 0 to 9039
Columns: 488 entries, p_5_1 to p_323
dtypes: float64(57), int64(431)
memory usage: 33.7 MB


In [ ]:
df_dicc.iloc[:,0:413] = df1.iloc[:, 0:413].astype(object)
df_dicc.head()

,p_5_1,p_5_2,p_6_1,p_6_2,p_6_3,p_6_4,p_6_5,p_6_6,p_6_7,p_6_8,...,p_314,p_315,p_316,p_317,p_318,p_319,p_320,p_321,p_322,p_323
0,1,0,0,0,1,0,0,0,0,0,...,4,4,4,2,4,4,4,5,4,4
1,1,0,0,0,0,0,0,1,0,0,...,4,4,4,4,4,4,4,4,4,4
2,1,0,0,0,0,0,0,0,0,0,...,3,4,4,4,4,4,4,4,4,4
3,1,0,0,0,0,0,0,0,0,0,...,4,4,2,3,4,4,4,4,4,4
4,1,0,0,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4


In [ ]:
df_dicc.iloc[:, 413:] = df1.iloc[:, 413:].astype(float)

In [ ]:
df_dicc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9040 entries, 0 to 9039
Columns: 488 entries, p_5_1 to p_323
dtypes: float64(76), object(412)
memory usage: 33.7+ MB


In [ ]:
datapoints = df_dicc.values
m, f = datapoints.shape
k = 3

In [ ]:
medoids_initial = init_medoids(datapoints, 3)

In [ ]:
S = compute_gower(datapoints, medoids_initial)

C:\ProgramData\Anaconda3\lib\site-packages\gower\gower_dist.py:51: RuntimeWarning: All-NaN slice encountered
  max = np.nanmax(col_array)
C:\ProgramData\Anaconda3\lib\site-packages\gower\gower_dist.py:52: RuntimeWarning: All-NaN slice encountered
  min = np.nanmin(col_array)


In [ ]:
np.savetxt('S', S)

In [ ]:
labels = assign_labels(S)

In [ ]:
start_time = time.time()

results = kmedoids(datapoints, 6, max_steps= 100)
final_medoids = results[0]
data = pd.DataFrame([])
data['clusters'] = results[1]

print("--- %s hours ---" % ((time.time() - start_time)/60*60))

--- 70971.38188123703 hours ---


In [ ]:
df_medoides = pd.DataFrame(results[0])
df_labels = pd.DataFrame(results[1])

In [ ]:
df_medoides

,0,1,2,3,4,5,6,7,8,9,...,478,479,480,481,482,483,484,485,486,487
0,1,0,0,0,0,0,0,0,0,0,...,2,4,4,3,4,4,4,4,4,4
1,1,0,0,0,0,0,0,0,0,0,...,3,3,4,3,3,3,3,4,4,4
2,1,0,0,0,0,1,0,0,0,0,...,3,4,4,4,4,4,4,4,4,4
3,1,0,0,0,0,0,0,1,0,0,...,2,4,4,4,4,4,4,3,4,4
4,1,0,0,0,1,0,0,0,0,0,...,4,4,4,2,4,4,4,5,4,4
5,1,0,0,1,0,0,0,0,0,0,...,3,4,4,4,4,4,2,4,4,4


In [ ]:
df_labels.head()

,0
0,4
1,4
2,2
3,4
4,2


In [ ]:
df_conk = df_dicc
df_conk['etiqueta_km'] = df_labels[0]

In [ ]:
df_conk.head()

,p_5_1,p_5_2,p_6_1,p_6_2,p_6_3,p_6_4,p_6_5,p_6_6,p_6_7,p_6_8,...,p_315,p_316,p_317,p_318,p_319,p_320,p_321,p_322,p_323,etiqueta_km
0,1,0,0,0,1,0,0,0,0,0,...,4.0,4.0,2.0,4.0,4.0,4.0,5.0,4.0,4.0,4
1,1,0,0,0,0,0,0,1,0,0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4
2,1,0,0,0,0,0,0,0,0,0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2
3,1,0,0,0,0,0,0,0,0,0,...,4.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4
4,1,0,0,0,0,0,0,0,0,0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2


In [ ]:
##df_conk.info()

In [ ]:
#df_conk[['p_7']] = df_conk[['p_7']].replace('AE2', '9014')
#df_conk[['p_7']] = df_conk[['p_7']].replace('AUC1', '9015')
#df_conk[['p_7']] = df_conk[['p_7']].replace('AE5', '9016')
#df_conk[['p_7']] = df_conk[['p_7']].replace('AE6', '9017')
#df_conk[['p_7']] = df_conk[['p_7']].replace('AE7', '9018')
#df_conk[['p_7']] = df_conk[['p_7']].replace('AUC2', '9019')

In [ ]:
df_conk.to_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Results/R2-K-med.csv', sep=';')

# Regresión lineal con *backward elimination*

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.feature_selection/

In [ ]:
def backward_elimination(data, target, significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

In [ ]:
df_conk['p_286_19'] = df_conk['p_286_19'].replace(np.nan, 0.0)
#df_be.iloc[:,0:52] = df_be.iloc[:,0:52].astype(float)
#df_be.info()

In [ ]:
ft = backward_elimination( df_conk.iloc[:,:-1].astype(float), df_labels.astype(float), significance_level = 0.05)

In [ ]:
ft = np.asarray(ft).astype(object)
ft

array(['p_5_1', 'p_5_2', 'p_6_2', 'p_6_3', 'p_6_4', 'p_6_9', 'p_6_12',
       'p_7_103', 'p_7_205', 'p_7_304', 'p_7_307', 'p_7_401', 'p_7_402',
       'p_7_403', 'p_7_404', 'p_7_405', 'p_7_407', 'p_7_408', 'p_7_409',
       'p_7_410', 'p_7_411', 'p_7_412', 'p_7_413', 'p_7_414', 'p_7_415',
       'p_7_507', 'p_7_510', 'p_7_602', 'p_7_603', 'p_7_604', 'p_7_605',
       'p_7_607', 'p_7_611', 'p_7_712', 'p_7_901', 'p_7_902', 'p_7_907',
       'p_7_908', 'p_7_911', 'p_7_916', 'p_7_917', 'p_7_1004', 'p_7_1206',
       'p_7_1211', 'p_7_1403', 'p_7_1416', 'p_7_1509', 'p_7_1510',
       'p_7_1615', 'p_7_1621', 'p_7_6003', 'p_10_1', 'p_10_2', 'p_10_3',
       'p_10_4', 'p_148_4', 'p_158_1', 'p_165_1', 'p_165_2', 'p_171_1',
       'p_171_2', 'p_174_1', 'p_174_2', 'p_178_1', 'p_178_2', 'p_284_1',
       'p_284_3', 'p_284_6', 'p_285_-88', 'p_12', 'p_15a', 'p_19b',
       'p_19d', 'p_19e', 'p_20b', 'p_36a', 'p_44', 'p_45b', 'p_45d',
       'p_66c', 'p_68b', 'p_86a', 'p_86b', 'p_86d', 'p_157', 'p_232

In [ ]:
X_opt = df_conk[ft]

In [ ]:
#X_opt.info()

In [ ]:
model = sm.OLS(df_labels.astype(float),X_opt.astype(float))
results_opt = model.fit()

In [ ]:
print(results_opt.summary())

                            OLS Regression Results                            
Dep. Variable:                      0   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     100.5
Date:                Wed, 24 Jun 2020   Prob (F-statistic):               0.00
Time:                        18:15:37   Log-Likelihood:                -13559.
No. Observations:                9040   AIC:                         2.731e+04
Df Residuals:                    8946   BIC:                         2.797e+04
Df Model:                          93                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
p_5_1          0.2326      0.025      9.154      0.0